In [1]:
# add path of modules
import sys
sys.path.append('d:\\Users\\serfe\\Documents\\GitHub\\seisinv\\src')

# Import local libraries
from basic_ops_seis_inv import MathOps, Filtering, Error, ConversionTool
from petro_analysis import DensityAnalysis
from post_stack import poststack_methods as ps_methods
from plot_seis_inv import PlotSeisInv

# Import standard libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
### Create instances to invoke functions/methods
math_ops = MathOps()
filtering = Filtering()
ps_methods = ps_methods()
plot_seis_inv = PlotSeisInv()
error = Error()
density_analysis = DensityAnalysis()
conversion_tool = ConversionTool()

In [3]:
# Read data
path_data = 'd:\\Users\\serfe\\Documents\\GitHub\\seisinv\\data\\data_task5.xlsx'
df = pd.read_excel(path_data)
df

,DEPT,Bit Size,Caliper,Compressional Slownes,Shear Slownes,Gamma Ray,Neutron,Density,Deep Resistivity,Spontaneous Potential,Surface
0,3000,9.5,9.07805,130.18320,287.5545,23.30912,0.56499,2.06083,0.44333,-45.88697,Top_1
1,3001,9.5,9.11115,132.22950,284.9733,27.90894,0.47970,2.07622,0.46744,-46.03849,Top_1
2,3002,9.5,9.12215,128.80150,282.1579,27.11216,0.48192,2.08354,0.48176,-46.62711,Top_1
3,3003,9.5,9.12217,128.46430,279.9890,25.41507,0.53129,2.08723,0.48818,-47.52837,Top_1
4,3004,9.5,9.05605,127.94190,278.7398,30.71320,0.47623,2.06726,0.49740,-47.99183,Top_1
...,...,...,...,...,...,...,...,...,...,...,...
5196,8196,9.5,8.74737,82.19551,179.6912,53.67944,0.22237,2.46533,5.03951,3.34816,Top_5
5197,8197,9.5,8.74731,77.29253,177.9962,59.17858,0.22924,2.44470,6.33363,7.78983,Top_5
5198,8198,9.5,8.70311,74.77816,176.1770,66.17214,0.26280,2.43656,7.77693,9.60661,Top_5
5199,8199,9.5,8.65917,75.38631,170.7982,67.56411,0.28504,2.49777,9.01924,9.85767,Top_5


In [4]:
# calculate impedance
rho, vp = conversion_tool.SI_conversion(df['Density'], 'g/cm3'), 1/conversion_tool.SI_conversion(df['Compressional Slownes'], 'micros/ft')
Z = math_ops.calc_impedance(rho, vp)
# add impedance to dataframe
df['Z'] = Z
df['VP'] = vp

Output units: kg/m3


In [5]:
# calculate vp-vs ratio
vs = 1/conversion_tool.SI_conversion(df['Shear Slownes'], 'micros/ft')
ratio = math_ops.calc_vp_vs_ratio(vp, vs)
# add Vs and vp/vs ratio
df['VS'] = vs
df['VP/VS'] = ratio

In [6]:
# calculate poisson coefficient
poisson = math_ops.calc_poisson(vp, vs)
df['poisson'] = poisson

In [7]:
# calculate mu and k modules
vp, vs, rho = df['VP'], df['VS'],  conversion_tool.SI_conversion(df['Density'], 'g/cm3')
mu = math_ops.calc_mu(vs, rho)
k = math_ops.calc_k(vp, vs, rho)
df['mu'] = mu
df['k'] = k

Output units: kg/m3


In [8]:
# Equivalent model exercise
vp = np.array([2.27, 2.5, 2.38, 2.4, 2.36])
vs = np.array([1.4, 1.5, 1.43, 1.4, 1.41])
rho = np.array([2.3, 2.16, 2.19, 2.38, 2.5])
lambda_lame, mu_lame = math_ops.calc_lame_parameters(rho, vp, vs)

C, F, L, M = math_ops.calc_coefficients_equiv_model(lambda_lame, mu_lame)
vp0, vs0, rho0 = math_ops.calc_equivalent_vel(C, L, rho)

In [9]:
vp0

2.3767721242313677

In [10]:
vs0

1.4258834715451107

In [13]:
rho0

2.306

In [12]:
M

4.6962762